<a href="https://colab.research.google.com/github/MasahiroAraki/MLCourse/blob/master/Python/12_apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第12章 パターンマイニング

## 頻出項目集合抽出

Aprioriアルゴリズムの実装

http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

In [ ]:
# colabで実行する場合、mlxtendを最新版に更新し、更新後に設定を読み直す
!pip install -U mlxtend
import pkg_resources, imp
imp.reload(pkg_resources)

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

例題12.1 のデータを読み込んで、pandasのDataFrameに変換。項目がアルファベット順にソートされていることに注意。

In [ ]:
dataset = [['Milk', 'Bread', 'Butter'],
  ['Milk', 'Bread', 'Jam'],
  ['Milk', 'Magazine'],
  ['Bread', 'Butter'],
  ['Milk', 'Bread', 'Butter', 'Jam'],
  ['Magazine'],
  ['Milk', 'Bread', 'Jam', 'Magazine'],
  ['Jam']]

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

Aprioriアルゴリズムで3事例以上出現する項目集合を抽出

In [ ]:
freq = apriori(df, min_support= 3/len(df), use_colnames=True)
freq

## 規則抽出

In [ ]:
association_rules(freq, metric="confidence", min_threshold=0.7)


supermarket.arffでFPGrowth（Aprioriの高速化版）を試してみます。データをダウンロードし、scipyのloadarffでWekaのarff形式のデータを読み込みます。その際、読み込みエラーを回避するために文字列置換を行い、最終的にpandasのDataFrameで値をTrue/Falseとし、最終列のtotalは除きます。

In [ ]:
!wget https://raw.githubusercontent.com/fracpete/wekamooc/master/dataminingwithweka/data/supermarket.arff

In [ ]:
!sed -e "s/{ t}/{t}/" supermarket.arff > supermarket2.arff

In [ ]:
from scipy.io import arff
data, meta = arff.loadarff('supermarket2.arff')
df = pd.DataFrame(data)
df2 = df.replace({b'?':False, b't':True})
df2 = df2.drop('total', axis=1)
df2

FPGrowthで頻出項目集合を求め、association_rulesで規則を抽出します。

In [ ]:
freq2 = fpgrowth(df2, min_support=0.3, use_colnames=True)

In [ ]:
association_rules(freq2, metric="lift", min_threshold=1.2)



## 行列分解

小規模な映画評価データを使って、行列分解を行います。



ライブラリの読み込み

In [ ]:
import numpy as np
from sklearn.decomposition import NMF

<a href="http://www.quuxlabs.com/blog/2010/09/matrix-factorization-a-simple-tutorial-and-implementation-in-python/">参考ページ</a>のデータ例を使用します。縦がユーザ、横が映画、数値が1-5の5段階評価で、0は評価なしです。

In [ ]:
X = np.array([
    [5,3,0,1],
    [4,0,0,1],
    [1,1,0,5],
    [1,0,0,4],
    [0,1,5,4]
])

ここでは、非負値行列分解NMFを使います。NMFはXを非負の行列WとHに分解します。引数n_componentsは潜在変数の次元数です。

In [ ]:
model = NMF(n_components = 2)
W = model.fit_transform(X)
H = model.components_

空所の値を予測してみます。似たようなユーザを参考に、埋められた値の妥当性を評価してみてください。

In [ ]:
np.set_printoptions(suppress=True)
np.dot(W,H)

人を表す2次元ベクトルを表示します。同じような評価をする人が似たベクトルになっていることを確認してください。

In [ ]:
W

映画を表す2次元ベクトルを表示します。同じような評価をされる映画がないので、それぞれ異なったベクトルになっていることを確認してください。

In [ ]:
H.T

## 課題1

アプリオリアルゴリズムによる規則抽出のコード例で求めた規則よりも、Lift値の高い規則を探すにはどうすればよいでしょうか。

## 課題2

教科書p.226の演習問題12-3の手順に従い、MovieLensデータセットに対してscikit-surpriseを用いてNMFを行ってください。余力があれば、ハイパーパラメータを変更して、性能に与える影響を観察してください。